# A Meeting Analysis Use Case

Copyright 2025, Denis Rothman

The notebook's goal is to demonstrate how to guide an AI through a multi-step analytical process, moving from a raw transcript to actionable insights, thereby training both the user and the AI.

This notebook is a step-by-step guide on how you can engineer a structured context . This guide will serve as the architectural blueprint for your code, explaining not just the *what* but the *why* at each stage.


-----

### Setup and Preliminaries

This section handles the basic configuration.

1.  **Cell 1: Install Libraries**

      * Install the necessary OpenAI library.

In [ ]:
# Cell 1: Installation
!pip install openai

2.  **Cell 2: Imports and API Key**

      * Import the library and securely prompt for the user's API key. This is better than hard-coding it.

In [ ]:
# Cell 2: Imports and API Key Setup
# We will use the OpenAI library to interact with the LLM and Google Colab's
# secret manager to securely access your API key.

import os
from openai import OpenAI
from google.colab import userdata

# Load the API key from Colab secrets, set the env var, then init the client
try:
    api_key = userdata.get("API_KEY")
    if not api_key:
        raise userdata.SecretNotFoundError("API_KEY not found.")

    # Set environment variable for downstream tools/libraries
    os.environ["OPENAI_API_KEY"] = api_key

    # Create client (will read from OPENAI_API_KEY)
    client = OpenAI()
    print("OpenAI API key loaded and environment variable set successfully.")

except userdata.SecretNotFoundError:
    print('Secret "API_KEY" not found.')
    print('Please add your OpenAI API key to the Colab Secrets Manager.')
except Exception as e:
    print(f"An error occurred while loading the API key: {e}")


OpenAI API key loaded and environment variable set successfully.


3.  **Cell 3: The Raw Data (The "Crime Scene")**

      * Define the meeting transcript as a multi-line string. This is our primary data source.

In [ ]:
# Cell 3: The Full Meeting Transcript
meeting_transcript = """
        Tom: Morning all. Coffee is still kicking in.
        Sarah: Morning, Tom. Right, let's jump in. Project Phoenix timeline. Tom, you said the backend components are on track?
        Tom: Mostly. We hit a small snag with the payment gateway integration. It's... more complex than the docs suggested. We might need another three days.
        Maria: Three days? Tom, that's going to push the final testing phase right up against the launch deadline. We don't have that buffer.
        Sarah: I agree with Maria. What's the alternative, Tom?
        Tom: I suppose I could work over the weekend to catch up. I'd rather not, but I can see the bind we're in.
        Sarah: Appreciate that, Tom. Let's tentatively agree on that. Maria, what about the front-end?
        Maria: We're good. In fact, we're a bit ahead. We have some extra bandwidth.
        Sarah: Excellent. Okay, one last thing. The marketing team wants to do a big social media push on launch day. Thoughts?
        Tom: Seems standard.
        Maria: I think that's a mistake. A big push on day one will swamp our servers if there are any initial bugs. We should do a soft launch, invite-only for the first week, and then do the big push. More controlled.
        Sarah: That's a very good point, Maria. A much safer strategy. Let's go with that. Okay, great meeting. I'll send out a summary.
        Tom: Sounds good. Now, more coffee.
        """

-----

### Layer 1: Establishing the scope (The 'What')

Here, we define the scope of the analysis. Each step's output will inform the next, creating a chain of context.

1.  **Cell 4: g2 - Set the Signal-to-Noise Ratio**

      * We'll start by cleaning the data. The prompt explicitly tells the AI to separate substantive content from conversational noise.

In [ ]:
# Cell 4: g2 - Isolating Content from Noise
prompt_g2 = f"""
        Analyze the following meeting transcript. Your task is to isolate the substantive content from the conversational noise.
        - Substantive content includes: decisions made, project updates, problems raised, and strategic suggestions.
        - Noise includes: greetings, pleasantries, and off-topic remarks (like coffee).
        Return ONLY the substantive content.

        Transcript:
        ---
        {meeting_transcript}
        ---
        """

In [ ]:
from openai import OpenAI

try:
    client = OpenAI()

    response_g2 = client.chat.completions.create(
        model="gpt-5",
        messages=[
            {"role": "user", "content": prompt_g2}
        ]
    )

    substantive_content = response_g2.choices[0].message.content
    print("--- SUBSTANTIVE CONTENT ---")
    print(substantive_content)

except Exception as e:
    print(f"An error occurred: {e}")

--- SUBSTANTIVE CONTENT ---
- Project Phoenix timeline: Backend mostly on track, but payment gateway integration is more complex than expected; needs an additional three days.
- Impact: Extra three days would push final testing up against the launch deadline, reducing buffer.
- Mitigation decision: Tom will work over the weekend to catch up (tentatively agreed).
- Front-end status: Ahead of schedule with extra bandwidth.
- Marketing/launch strategy: Initial plan for a big social media push on launch day flagged as risky (potential server load with early bugs). Decision: Use a soft launch (invite-only) for the first week, then execute the big push.


2.  **Cell 5: g3 - Define the Scope of Time (Simulated RAG)**

      * We'll simulate a RAG context by providing a "previous" summary and asking for what's new. This teaches the user the importance of historical context.

In [ ]:
# Cell 5: g3 - Identifying NEW Information (Simulated RAG)
previous_summary = "In our last meeting, we finalized the goals for Project Phoenix and assigned backend work to Tom and front-end to Maria."

prompt_g3 = f"""
Context: The summary of our last meeting was: "{previous_summary}"

Task: Analyze the following substantive content from our new meeting. Identify and summarize ONLY the new developments, problems, or decisions that have occurred since the last meeting.

New Meeting Content:
---
{substantive_content}
---
"""
from openai import OpenAI
from google.colab import userdata


# Your chat completion request
try:
    response_g3 = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt_g3}]
    )
    new_developments = response_g3.choices[0].message.content
    print("--- NEW DEVELOPMENTS SINCE LAST MEETING ---")
    print(new_developments)
except Exception as e:
    print(f"An error occurred: {e}")


--- NEW DEVELOPMENTS SINCE LAST MEETING ---
- Backend issue: Payment gateway integration is more complex than expected; needs an additional three days.
- Schedule impact: The extra three days compress final testing, pushing it up against the launch deadline and reducing buffer.
- Mitigation decision: Tentative agreement that Tom will work over the weekend to catch up.
- Front-end status: Ahead of schedule with extra bandwidth.
- Launch/marketing decision: Shift from a big day-one social push to a one-week invite-only soft launch, followed by the major push.


-----

### Layer 2: Conducting the Investigation (The 'How')

Now we move from identifying facts to generating insights, the core of the semantic context interpreation journey.

1.  **Cell 6: g4 - Identify the Key Threads**

      * This is a crucial step. The prompt asks the AI to read between the lines.

In [ ]:
# Cell 6: g4 - Uncovering Implicit Threads
prompt_g4 = f"""
Task: Analyze the following meeting content for implicit social dynamics and unstated feelings. Go beyond the literal words.
- Did anyone seem hesitant or reluctant despite agreeing to something?
- Were there any underlying disagreements or tensions?
- What was the overall mood?

Meeting Content:
---
{substantive_content}
---
"""
try:
    response_g4 = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt_g4}]
    )
    implicit_threads = response_g4.choices[0].message.content
    print("--- IMPLICIT THREADS AND DYNAMICS ---")
    print(implicit_threads)
except Exception as e:
    print(f"An error occurred: {e}")

--- IMPLICIT THREADS AND DYNAMICS ---
Here’s what seems to be happening beneath the surface:

Hesitation/reluctance despite agreement
- Tom’s “tentative” agreement to work over the weekend reads as reluctant. It suggests he felt pressure to volunteer rather than genuine willingness.
- Marketing likely agreed to the soft launch with some reluctance; shifting from a big day-one push to invite-only implies a concession to engineering risk.

Underlying disagreements or tensions
- Pace vs quality: Engineering wants stability and buffer; marketing originally aimed for impact. The soft launch is a compromise, but the differing risk appetites remain.
- Workload equity: Backend is behind while frontend has “extra bandwidth.” The decision to have Tom work the weekend (vs redistributing tasks) hints at siloing or a norm of individual heroics, which can breed quiet resentment.
- Testing squeeze: Pushing testing against the deadline implies QA will be under pressure, potentially creating friction i

2.  **Cell 7: g5 - Perform "Intellectual Combinations"**

      * Here, we prompt the AI to be creative and solve a problem by synthesizing different ideas from the meeting.

In [ ]:
# Cell 7: g5 - Generating a Novel Solution
prompt_g5 = f"""
Context: In the meeting, Maria suggested a 'soft launch' to avoid server strain, and also mentioned her team has 'extra bandwidth'.
Tom is facing a 3-day delay on the backend.

Task: Propose a novel, actionable idea that uses Maria's team's extra bandwidth to help mitigate Tom's 3-day delay. Combine these two separate pieces of information into a single solution.
"""
try:
    response_g5 = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt_g5}]
    )
    novel_solution = response_g5.choices[0].message.content
    print("--- NOVEL SOLUTION PROPOSED BY AI ---")
    print(novel_solution)
except Exception as e:
    print(f"An error occurred: {e}")

--- NOVEL SOLUTION PROPOSED BY AI ---
Idea: Soft launch behind a temporary “Edge Bridge” that Maria’s team builds to buffer reads/writes until Tom’s backend is ready.

What Maria’s team does (uses their extra bandwidth)
- Stand up a thin serverless/API facade that matches the real API contracts (e.g., API Gateway/Lambda + SQS/DynamoDB or Cloudflare Workers + Durable Objects).
- Reads: Serve from a prewarmed cache or static snapshots (stale-while-revalidate). Update snapshots hourly via a lightweight data export from staging or existing endpoints.
- Writes: Capture requests into a durable queue with idempotency keys; return immediate “queued” success to the client and show “syncing” UI. When Tom’s backend is live, a replay worker drains the queue and applies changes.
- Add feature flags/traffic gating (e.g., LaunchDarkly) to limit the soft launch cohort and throttle requests to avoid server strain.

How this mitigates the 3-day delay
- The product can soft-launch to a small cohort witho

-----

### Layer 3: Determining the Action (The 'What Next')

Finally, we turn the analysis into concrete, forward-looking artifacts.

1.  **Cell 8: g6 - Define the Output Format (Final Summary)**

      * We compile all the key information into a structured, final summary, showing the importance of clear outputs.

In [ ]:
# Cell 8: g6 - Creating the Final, Structured Summary
prompt_g6 = f"""
Task: Create a final, concise summary of the meeting in a markdown table.
Use the following information to construct the table.

- New Developments: {new_developments}

The table should have three columns: "Topic", "Decision/Outcome", and "Owner".
"""
try:
    response_g6 = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt_g6}]
    )
    final_summary_table = response_g6.choices[0].message.content
    print("--- FINAL MEETING SUMMARY TABLE ---")
    print(final_summary_table)
except Exception as e:
    print(f"An error occurred: {e}")

--- FINAL MEETING SUMMARY TABLE ---
| Topic | Decision/Outcome | Owner |
|---|---|---|
| Backend payment gateway integration | More complex than expected; requires an additional three days | Backend Team |
| Schedule impact | Extra three days compress final testing, reducing buffer before launch | Project Manager |
| Mitigation | Tentative plan: Tom will work over the weekend to catch up | Tom |
| Front-end status | Ahead of schedule with extra bandwidth available | Front-end Team |
| Launch/marketing plan | Shift to a one-week invite-only soft launch, then major day-one push | Marketing + Product |


2.  **Cell 9: g7 - Generate the Subsequent Task**

      * The last step is to use the analysis to perform a real-world action, closing the loop from insight to action.

In [ ]:
# Cell 9: g7 - Drafting the Follow-Up Action
prompt_g7 = f"""
Task: Based on the following summary table, draft a polite and professional follow-up email to the team (Sarah, Tom, Maria).
The email should clearly state the decisions made and the action items for each person.

Summary Table:
---
{final_summary_table}
---
"""
try:
    response_g7 = client.chat.completions.create(
        model="gpt-5",
        messages=[{"role": "user", "content": prompt_g7}]
    )
    follow_up_email = response_g7.choices[0].message.content
    print("--- DRAFT FOLLOW-UP EMAIL ---")
    print(follow_up_email)
except Exception as e:
    print(f"An error occurred: {e}")

--- DRAFT FOLLOW-UP EMAIL ---
Subject: Follow-up: Decisions and next steps from today’s sync

Hi Sarah, Tom, and Maria,

Thanks for the productive discussion earlier. Here’s a quick recap of decisions and the action items for each of us.

Decisions
- Backend payment gateway integration is more complex than expected and will require an additional three days.
- This pushes the schedule by three days and compresses the final testing window, reducing our pre-launch buffer.
- Mitigation: Tom will work over the weekend to help us catch up.
- Front-end is ahead of schedule and has extra bandwidth to support.
- Launch/marketing plan will shift to a one-week invite-only soft launch, followed by the larger day-one push.

Action items
- Tom:
  - Confirm weekend availability and share a brief plan (key milestones, dependencies, and any risks).
  - Proceed with the gateway integration and coordinate early integration testing with Front-end and QA.
  - Provide short daily progress updates and flag b

This notebook structure doesn't just "get a summary." It takes the user on a journey, showing them *how* to think with the AI as a partner. It perfectly translates the abstract "Scope, Investigation, Action" framework into a repeatable, educational, and powerful process.